In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [ ]:
# Load datasets
# census_data = pd.read_csv('..''Resources'') 
## ^^ this will become API right? -natalie 
all_median_prices = pd.read_csv('..''Resources''median_sales_price_all.csv')
sfr_median_prices = pd.read_csv('..''Resources''median_sales_price_sfr.csv')

# Display the first few rows of each dataset
census_data.head()
real_estate_data.head()
sales_data.head()

In [ ]:
# Get summary statistics
census_data.info()
real_estate_data.info()
sales_data.info()

# Check for missing values
census_data.isnull().sum()
real_estate_data.isnull().sum()
sales_data.isnull().sum()

In [ ]:
# Example cleaning steps (these will vary based on actual data issues found)

# Handle missing values (e.g., drop rows with missing target values)
real_estate_data = real_estate_data.dropna(subset=['price'])

# Convert date columns to datetime
sales_data['date'] = pd.to_datetime(sales_data['date'])

# Standardize numerical formats
census_data['income'] = census_data['income'].astype(float)

# Filter data for Nashville metro area
nashville_data = real_estate_data[real_estate_data['metro_area'] == 'Nashville']

In [ ]:
# Merge datasets on common keys
merged_data = pd.merge(nashville_data, census_data, on='common_key')
merged_data = pd.merge(merged_data, sales_data, on='common_key')

# Verify the merged dataset
merged_data.head()
merged_data.info()